# A3. Loading AP Sensing files
This example loads AP sensing files. Only single-ended files are currently supported. 

The currently supported AP Sensing N4386B device has two data logging options to log into .xml files and .tra files. Only .xml files contain the stokes and anti-stokes intensities needed for this calibration. Unfortunately, these .xml files are scarce on metadata and do not contain the additionally connected sensors e.g. PT100 from the device. The latter are contained inside the .tra file.

If you did not connect any additional sensors, you can use .xml files only and add your own logged temperature data to the datastore for calibration. (Hint: The .xml file export is well hidden in your AP Sensing software *DTS Configurator* and not documented in the user manual. Inside your *Configuration* turn *POSC export* on - this will export the .xml file.)

If you want to additionally use data exported to .tra files (e.g. PT100 data) use the .tra logging make sure to enable *Auto Save Traces* in under *Program Options* and make sure *Create Multitrace files* and *Use Binary Format* are both disabled. Make sure to place the .tra files into the identical directory as the .xml files. Then they will be imported automatically with the *read_apsensing_files* commmand.

The current implementation of .tra file parsing is limited to in-memory reading only.

In [12]:
import os
import glob

from dtscalibration import read_apsensing_files

The example data files are located in `./python-dts-calibration/tests/data`.

In [13]:
filepath = os.path.join("..", "..", "tests", "data", "ap_sensing")
filepath_with_tra = os.path.join("..", "..", "tests", "data", "ap_sensing_2", "CH1_SE")
print(filepath)

..\..\tests\data\ap_sensing


In [14]:
filepathlist = sorted(glob.glob(os.path.join(filepath, "*.xml")))
filenamelist = [os.path.basename(path) for path in filepathlist]

for fn in filenamelist:
    print(fn)

_AP Sensing_N4386B_3_20180118201727.xml
_AP Sensing_N4386B_3_20180118202957.xml
_AP Sensing_N4386B_3_20180118205357.xml


We will simply load in the .xml files

In [15]:
ds = read_apsensing_files(directory=filepath)
ds_with_tra = read_apsensing_files(directory=filepath_with_tra)

3 files were found, each representing a single timestep
4 recorded vars were found: LAF, TEMP, ST, AST
Recorded at 7101 points along the cable
The measurement is single ended
Reading the data from disk
3 files were found, each representing a single timestep
4 recorded vars were found: LAF, TEMP, ST, AST
Recorded at 1201 points along the cable
The measurement is single ended
Reading the data from disk
.tra files exist and will be read


C:\Users\David Lah\Documents\dts-data-processing\extern\python-dts-calibration\src\dtscalibration\io\apsensing.py:480: UserWarning: Not all .xml files have a matching .tra file.
 Missing are time following timestamps {'20180118202957', '20180118201727', '20180118205357'}.  Not loading .tra data.
  warnings.warn(msg)


The object tries to gather as much metadata from the measurement files as possible (temporal and spatial coordinates, filenames, temperature probes measurements). All other configuration settings are loaded from the first files and stored as attributes of the `xarray.Dataset`.
y
Calibration follows as usual (see the other notebooks).

In [16]:
print(ds)

<xarray.Dataset> Size: 569kB
Dimensions:       (x: 7101, time: 3)
Coordinates:
  * x             (x) float64 57kB 0.0 0.5 1.0 ... 3.549e+03 3.55e+03 3.55e+03
    filename      (time) <U39 468B '_AP Sensing_N4386B_3_20180118201727.xml' ...
  * time          (time) datetime64[ns] 24B 2018-01-18T20:17:27 ... 2018-01-1...
Data variables:
    tmp           (x, time) float64 170kB 12.16 11.32 12.26 ... 15.08 17.83
    st            (x, time) float64 170kB 1.098 1.105 ... 3.39e-18 3.409e-18
    ast           (x, time) float64 170kB 0.1888 0.1891 ... 4.838e-19 4.945e-19
    creationDate  (time) datetime64[ns] 24B 2018-01-18T20:17:27 ... 2018-01-1...
Attributes: (12/51)
    wellbore:uid:                                                            ...
    wellbore:name:                                                           ...
    wellbore:dtsInstalledSystemSet:dtsInstalledSystem:uid:                   ...
    wellbore:dtsInstalledSystemSet:dtsInstalledSystem:name:                  ...
    we

In [17]:
print(ds_with_tra.probe1Temperature)
print(ds_with_tra)

<xarray.DataArray 'probe1Temperature' (time: 3)> Size: 24B
array([19.60636, 19.62306, 19.62306])
Coordinates:
    filename  (time) <U45 540B 'CH1_SE_AP Sensing_N4386B_1_20240130141820.xml...
  * time      (time) datetime64[ns] 24B 2024-01-30T14:18:20 ... 2024-01-30T14...
<xarray.Dataset> Size: 97kB
Dimensions:            (x: 1201, time: 3)
Coordinates:
  * x                  (x) float64 10kB -50.0 -49.75 -49.5 ... 249.5 249.8 250.0
    filename           (time) <U45 540B 'CH1_SE_AP Sensing_N4386B_1_202401301...
  * time               (time) datetime64[ns] 24B 2024-01-30T14:18:20 ... 2024...
Data variables:
    tmp                (x, time) float64 29kB 22.49 22.85 23.14 ... 20.3 19.71
    st                 (x, time) float64 29kB 1.254 1.256 ... 0.8482 0.8397
    ast                (x, time) float64 29kB 0.2453 0.2461 ... 0.163 0.1609
    creationDate       (time) datetime64[ns] 24B 2024-01-30T14:18:20 ... 2024...
    probe1Temperature  (time) float64 24B 19.61 19.62 19.62
    probe2Tem